In [ ]:
import pandas as pd

from Server_Cleaning.modules.Assignment_datasets import *

path = "./"

In [ ]:
cg_g = clean_club_games(get_club_games(path), path)
club_games = cg_g[0]
games = cg_g[1]
print("doubles per match: ", ((club_games.drop_duplicates("game_id").shape[0] * 2) == club_games.shape[0]))
top_clubs_g = club_games.value_counts('club_id').reset_index().head(10)
clubs = clean_clubs(get_clubs(path))
top_10_clubs = clubs.query('club_id.isin(@top_clubs_g.club_id)', engine='python')
top_clubs_g

 `top_club_g` sono i 10 clubs con una maggiore mole di dati di partite a nostra disposizione

# --------------------------------------------------------------------------------
# Players

In [ ]:
players = clean_players(get_players(path))
player_val = clean_player_valuations(get_player_valuations(path))
appear = clean_appearances(get_appearances(path), games, path)

In [ ]:
print(players.columns)
print(player_val.columns)
appear.columns

In [ ]:
players = players[players['last_season'] > 2021]
player_val = player_val[player_val['last_season'] > 2021]
appear = appear[appear['game_date'] > pd.to_datetime("2021-12-31")]

In [ ]:
appear.head()

In [ ]:
player_val.sort_values(by='date')
top_player_vals = player_val.drop_duplicates('player_id', keep='last').sort_values(by='market_value_eur', ascending=False).head(10)
top_player_vals

In [ ]:
top_10_players = players.loc[players['player_id'].isin(top_player_vals['player_id'])]
top_10_players

In [ ]:
df1 = player_val.drop_duplicates('player_id', keep='last').sort_values(by='market_value_eur', ascending=False).head(50)
info_about_top_payed = players.loc[players['player_id'].isin(df1['player_id'])]
info_about_top_payed = pd.merge(info_about_top_payed, df1[['player_id', 'market_value_eur', 'date_week', 'current_dom_competition_code']], on='player_id', how='left')
info_about_top_payed = info_about_top_payed.drop(columns=['image_url'])
info_about_top_payed

In [ ]:
poor_players = players[~players['player_id'].isin(info_about_top_payed['player_id'])]
print(players.shape)
poor_players

In [ ]:
players.value_counts('height_in_cm').reset_index()

In [ ]:
countries = info_about_top_payed.groupby('country_of_citizenship').agg(count=pd.NamedAgg('country_of_citizenship', 'count')).reset_index()
countries #no good

In [ ]:
top_10_clubs.info()

# goals by minutes

In [ ]:
game_events = pd.read_csv("./Assignment_Data_2023-2024/game_events.csv")
game_events = game_events[game_events['type'] == "Goals"]
game_events['date'] = game_events['date'].astype('datetime64[ns]')
game_events = game_events[game_events['date'] > pd.to_datetime("2021-12-31")]
top_payed_game_events = game_events[game_events['player_id'].isin(info_about_top_payed['player_id'])]

In [ ]:
game_events = game_events.drop(columns=['date', 'game_event_id', 'game_id', 'type', 'club_id', 'description', 'player_in_id', 'player_assist_id'])
top_payed_game_events

goals by minutes achieved by every player since 2022

In [ ]:
game_events.set_index('player_id')
game_events.value_counts('player_id')
last_players = players[players['last_season']>=2022]
last_players_game_events = game_events[game_events['player_id'].isin(last_players['player_id'])]
last_players_game_events.value_counts('minute').sort_values(ascending=False)